### Installation
Install the packages required for executing this notebook.

In [1]:
# Install the packages
! pip3 install --user --no-cache-dir --upgrade "kfp>2" "google-cloud-pipeline-components>2" \
                                        google-cloud-aiplatform

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 595.6/595.6 kB 28.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


## Restart the kernel
Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

In [13]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Check the versions of the packages you installed. The KFP SDK version should be >2.

In [1]:
! python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"
! pip3 freeze | grep aiplatform
! python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"

KFP SDK version: 2.7.0
google-cloud-aiplatform==1.70.0
google_cloud_pipeline_components version: 2.17.0


In [2]:
import kfp
import typing
from typing import Dict
from typing import NamedTuple
from kfp import dsl
from kfp.dsl import (Artifact,
                        Dataset,
                        Input,
                        Model,
                        Output,
                        Metrics,
                        ClassificationMetrics,
                        component, 
                        OutputPath, 
                        InputPath)
import google.cloud.aiplatform as aip
from google_cloud_pipeline_components.types import artifact_types

#### Project and Pipeline Configurations

In [8]:
#The Google Cloud project that this pipeline runs in.
PROJECT_ID = "de2024-438710"
# The region that this pipeline runs in
REGION = "us-central1"
# Specify a Cloud Storage URI that your pipelines service account can access. The artifacts of your pipeline runs are stored within the pipeline root.
PIPELINE_ROOT = "gs://temp_de_a1"

#### Pipeline Component : Data Ingestion

In [9]:
@dsl.component(
    packages_to_install=["pandas","google-cloud-storage"],
    base_image="python:3.10.7-slim"
)
def download_data(project_id: str, bucket: str, file_name: str, dataset: Output[Dataset]):
    '''download data'''
    from google.cloud import storage
    import pandas as pd
    import logging 
    import sys
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    # Downloaing the file from a google bucket 
    client = storage.Client(project=project_id)
    bucket = client.bucket(bucket)
    blob = bucket.blob(file_name)
    blob.download_to_filename(dataset.path + ".csv")
    logging.info('Downloaded Data!')

#### Pipeline Component : Training-MLP 

In [10]:
@dsl.component(
    packages_to_install=['pandas', 'keras', 'tensorflow', 'h5py', 'scikit-learn'],
    base_image="python:3.10.7-slim"
)
def train_mlp (features: Input[Dataset], out_model: Output[Model]) -> NamedTuple('outputs', metrics=dict):
    '''train a MLP with default parameters'''
    import pandas as pd
    from keras.layers import Dense, Dropout, BatchNormalization
    from keras.models import Sequential
    from sklearn.preprocessing import StandardScaler
    from sklearn.model_selection import train_test_split
    from tensorflow.keras.callbacks import EarlyStopping
    import tensorflow as tf
    import json
    import logging 
    import sys
    import os
        
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    df = pd.read_csv(features.path+".csv")
    
    logging.info(df.columns)
        
    # split into input (X) and output (y) variables
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]

    # Split and scale the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Build the MLP model
    model = Sequential()
    model.add(Dense(128, input_dim=X_train_scaled.shape[1], activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    # Train the model
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_test_scaled, y_test), callbacks=[early_stopping])

    # evaluate the model
    scores = model.evaluate(X_test_scaled, y_test, verbose=0)
    logging.info(model.metrics_names)
    metrics_dict = {
        "accuracy": scores[1],
        "loss": scores[0],
    }   
    logging.info(metrics_dict)   
    
    out_model.metadata["file_type"] = ".h5"
    out_model.metadata["algo"] = "mlp"
    # Save the model
    m_file = out_model.path + ".h5"
    model.save(m_file)
    outputs = NamedTuple('outputs', metrics=dict)
    return outputs(metrics_dict)

#### Pipeline Component : Training LogisticRegression

In [11]:
@dsl.component(
    packages_to_install=['pandas', 'scikit-learn'],
    base_image="python:3.10.7-slim"
)
def train_lr (features: Input[Dataset], out_model: Output[Model]) -> NamedTuple('outputs', metrics=dict):
    '''train a LogisticRegression with default parameters'''
    import pandas as pd
    from sklearn.linear_model import LogisticRegression
    from sklearn import metrics
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler
    import json
    import logging 
    import sys
    import os
    import pickle  
       
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    df = pd.read_csv(features.path+".csv")
    
    logging.info(df.columns)        
    
    # Load the dataset
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]

    # Split and scale the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Train Logistic Regression model
    model = LogisticRegression()
    model.fit(X_train_scaled, y_train)

    metrics_dict = {
        "accuracy": model.score(X_test_scaled, y_test)
    }
    logging.info(metrics_dict)  
    
    out_model.metadata["file_type"] = ".pkl"
    out_model.metadata["algo"] = "lr"
    
    # Save the model
    m_file = out_model.path + ".pkl"
    with open(m_file, 'wb') as f:  
        pickle.dump(model, f)   
    
    outputs = NamedTuple('outputs', metrics=dict)
    return outputs(metrics_dict)

#### Pipeline Component : Prediction-MLP

In [12]:
@dsl.component(
    packages_to_install=['pandas', 'keras', 'tensorflow', 'h5py', 'scikit-learn'],
    base_image="python:3.10.7-slim"
)
def predict_mlp(model: Input[Model], features: Input[Dataset], results: Output[Dataset]):
    import pandas as pd
    from keras.models import load_model
    from sklearn.preprocessing import StandardScaler
    from sklearn.model_selection import train_test_split
    import json
    import logging
    import sys
    import os
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    # Load the dataset
    df = pd.read_csv(features.path + ".csv")
    
    # Split into input (X) and output (y) variables
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
    
    # Scale the data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Load the saved MLP model
    model_mlp = load_model(model.path + '.h5')
    
    # Make predictions using the MLP model on the test set
    result = model_mlp.predict(X_test_scaled)
    
    # For binary classification, apply a threshold to the output probabilities
    y_classes = (result > 0.5).astype(int)
    logging.info(y_classes)
    
    # Create a copy of the test set (so the number of rows matches the predictions)
    dfcp = X_test.copy()
    
    # Add the predicted class to the test set copy
    dfcp['pclass'] = y_classes.tolist()
    
    # Save the test set with predictions to a CSV file
    dfcp.to_csv(results.path + ".csv", index=False, encoding='utf-8-sig')

#### Pipeline Component : Prediction-LR

In [13]:
@dsl.component(
    packages_to_install=['pandas', 'scikit-learn'],
    base_image="python:3.10.7-slim"
)
def predict_lr(model: Input[Model], features: Input[Dataset], results: Output[Dataset]):
    import pandas as pd
    from sklearn.preprocessing import StandardScaler
    from sklearn.model_selection import train_test_split
    import pickle  
    import json
    import logging
    import sys
    import os
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    # Load the dataset
    df = pd.read_csv(features.path + ".csv")
    
    # Load the saved logistic regression model
    filename = model.path + ".pkl"
    model_lr = pickle.load(open(filename, 'rb'))
    
    # Split into input (X) and output (y) variables
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
    
    # Scale the data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Make predictions on the test set
    y_classes = model_lr.predict(X_test_scaled)
    logging.info(y_classes)
    
    # Create a copy of the test set
    dfcp = X_test.copy()
    dfcp['pclass'] = y_classes.tolist()

    # Save the test set with predictions to a CSV file
    dfcp.to_csv(results.path + ".csv", index=False, encoding='utf-8-sig')

#### Pipeline Component : Algorithm Selection 

In [14]:
@dsl.component(
    base_image="python:3.10.7-slim"
)
def compare_model(mlp_metrics: dict, lr_metrics: dict) -> str:
    import logging
    import json
    import sys
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    logging.info(mlp_metrics)
    logging.info(lr_metrics)
    if mlp_metrics.get("accuracy") > lr_metrics.get("accuracy"):
        return "MLP"
    else :
        return "LR"

### Upload Model and Metrics to Google Bucket 

In [15]:
@dsl.component(
    packages_to_install=["google-cloud-storage"],
    base_image="python:3.10.7-slim"
)
def upload_model_to_gcs(project_id: str, model_repo: str, model: Input[Model]):
    '''upload model to gsc'''
    from google.cloud import storage   
    import logging 
    import sys
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)    
  
    # upload the model to GCS
    client = storage.Client(project=project_id)
    bucket = client.bucket(model_repo)
    blob = bucket.blob(str(model.metadata["algo"]) + '_model' + str(model.metadata["file_type"])) 
    blob.upload_from_filename(model.path + str(model.metadata["file_type"]))       
    
    print("Saved the model to GCP bucket : " + model_repo)

#### Define the Pipeline

In [16]:
# Define the workflow of the pipeline.
@kfp.dsl.pipeline(
    name="heart-disease-predictor-training-pipeline")
def pipeline(project_id: str, data_bucket: str, trainset_filename: str, model_repo: str):
    
    
    di_op = download_data(
        project_id=project_id,
        bucket=data_bucket,
        file_name=trainset_filename
    )

 
    training_mlp_job_run_op = train_mlp(
        features=di_op.outputs["dataset"]
    )
    
     
    training_lr_job_run_op = train_lr(
        features=di_op.outputs["dataset"]
    )
        
        
    comp_model__op = compare_model(mlp_metrics=training_mlp_job_run_op.outputs["metrics"],
                                       lr_metrics=training_lr_job_run_op.outputs["metrics"]).after(training_mlp_job_run_op, training_lr_job_run_op)  
    
    # defining the branching condition
    with dsl.If(comp_model__op.output=="MLP"):
        predict_mlp_job_run_op = predict_mlp(
            model=training_mlp_job_run_op.outputs["out_model"],      
            features=di_op.outputs["dataset"]
        )
        upload_model_mlp_to_gc_op = upload_model_to_gcs(
            project_id=project_id,
            model_repo=model_repo,
            model=training_mlp_job_run_op.outputs['out_model']
        ).after(predict_mlp_job_run_op)
        
    with dsl.If(comp_model__op.output=="LR"):
        predict_lr_job_run_op = predict_lr(
            model=training_lr_job_run_op.outputs["out_model"],     
            features=di_op.outputs["dataset"]
        )
        upload_model_lr_to_gc_op = upload_model_to_gcs(
            project_id=project_id,
            model_repo=model_repo,
            model=training_lr_job_run_op.outputs['out_model']
        ).after(predict_lr_job_run_op)  

#### Compile the pipeline into a JSON file

In [22]:
from kfp import compiler
compiler.Compiler().compile(pipeline_func=pipeline,
        package_path='heart_disease_predictor_training_pipeline.yaml')

#### Submit the pipeline run

In [23]:
import google.cloud.aiplatform as aip

# Before initializing, make sure to set the GOOGLE_APPLICATION_CREDENTIALS
# environment variable to the path of your service account.
aip.init(
    project=PROJECT_ID,
    location=REGION,
)

# Prepare the pipeline job
job = aip.PipelineJob(
    display_name="heart-disease-predictor",
    enable_caching=False,
    template_path="heart_disease_predictor_training_pipeline.yaml",
    pipeline_root=PIPELINE_ROOT,
    location=REGION,
    parameter_values={
        'project_id': PROJECT_ID, # makesure to use your project id 
        'data_bucket': 'data_de_a1',  # makesure to use your data bucket name 
        'trainset_filename': 'data.csv',     # makesure to upload these to your data bucket
        'model_repo':'models_de_a1' # makesure to use your model bucket name 
    }
)

job.run()

Creating PipelineJob
PipelineJob created. Resource name: projects/430561060358/locations/us-central1/pipelineJobs/heart-disease-predictor-training-pipeline-20241024200509
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/430561060358/locations/us-central1/pipelineJobs/heart-disease-predictor-training-pipeline-20241024200509')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/heart-disease-predictor-training-pipeline-20241024200509?project=430561060358
PipelineJob projects/430561060358/locations/us-central1/pipelineJobs/heart-disease-predictor-training-pipeline-20241024200509 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/430561060358/locations/us-central1/pipelineJobs/heart-disease-predictor-training-pipeline-20241024200509 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/430561060358/locations/us-central1/pipelineJobs/heart-disease-predictor-